## MLE Estiamation 

$w = (X^T\cdot X)^{-1}\cdot X^{T}\cdot y$, because the cost function $||Xw - y||^{2} $

## MAP Estimation Overview
MAP estimation in regression involves incorporating a prior belief about the distribution of model parameters (in this case, the polynomial coefficients) and updating this belief based on the observed data. 

For polynomial regression, if we assume a Gaussian prior for the coefficients, the MAP estimation modifies the objective function used in the regression to include a regularization term that reflects this prior.

## Key Components of MAP in Polynomial Regression:

#### 1. Prior Distribution: You have a Gaussian prior on the polynomial coefficients 𝑤, typically expressed as:

$$𝑝(𝑤∣𝛼)=𝑁(0,𝛼^{−1}𝐼)$$

Here, $𝛼$ is the precision of the prior (not the variance). It reflects how strongly you believe the coefficients should be close to zero (larger 𝛼 implies stronger belief in smaller magnitude weights).

#### 2. Likelihood Function: This is the likelihood of observing the data given the model parameters. 

In linear regression, assuming Gaussian noise, the likelihood for a given set of parameters 𝑤 is also Gaussian centered around the model predictions. 

#### 3. Posterior Distribution: MAP estimates the coefficients 𝑤 by maximizing the posterior distribution, which, due to Bayes' Rule, is proportional to the product of the likelihood and the prior:

$$𝑝(𝑤∣𝑡,𝑋,𝛼,𝛽)∝𝑝(𝑡∣𝑋,𝑤,𝛽)×𝑝(𝑤∣𝛼)p(w∣t,X,α,β)∝p(t∣X,w,β)×p(w∣α)$$ 

where 𝛽 is the precision of the likelihood (related to the noise variance).

#### 4. Regularized Objective Function: In practical terms, maximizing the posterior is equivalent to minimizing a loss function that adds a regularization term to the usual sum of squared residuals:

$$Minimize: ∣∣𝑋𝑤−𝑡∣∣^{2}+𝛼∣∣𝑤∣∣^{2}$$
 
The regularization term $𝛼∣∣𝑤∣∣^{2}$ comes directly from the log of the Gaussian prior and penalizes large coefficients, effectively controlling overfitting.

## Steps for Bayesian Regression

### 1. Prior Distribution: 

Assume a Gaussian prior for the weights 𝑤 similar to MAP, 

$$𝑝(𝑤∣𝛼)=𝑁(0,𝛼^{−1}𝐼)$$

### 2. Likelihood: 

The likelihood remains Gaussian centered around the model predictions, 

$$p(t∣X,w,β)$$

### 3. Posterior Distribution: 

Compute the posterior distribution over the weights, 

$$p(w∣t,X,α,β)$$

which is also Gaussian but with updated mean and covariance based on the data:

$$𝑤∣𝑡,X∼N(m_{N} ,S_{N})$$

where $𝑚_{𝑁}$ and $S_{N}$ are updated based on the data and the priors.

### 4. Predictive Distribution: 

For new input $𝑥^{∗}$, the predictive distribution of the target $𝑡^{∗}$ is also Gaussian:

$$𝑡^{∗}∣𝑥^{∗},𝑡,𝑋∼𝑁(𝑚_{𝑁}^{𝑇} 𝜙(x^{∗}),𝜎_{𝑁}^{2}(𝑥^{∗}))$$

where $𝜙(^{∗})$ are the polynomial features of $x^{∗}$ and $𝜎_{𝑁}^{2}(𝑥^{∗})$ includes the variance from the model uncertainty.

In [ ]:
import numpy as np
from numpy.random import normal, uniform
from scipy.stats import multivariate_normal as mv_norm
import matplotlib.pyplot as plt

## Set the random seed
np.random.seed(0)
## Function to generate polynomial features
def polynomial_features(x, order):
    """ Generate polynomial features up to the given order """
    features = np.vander(x, N=order+1, increasing=True)
    return features

## Function to perform MLE Regression
def mle_regression(x, t, order):
    """ Perform MLE regression using polynomial features, when you calculate the w, the prior distribution is not considered """
    ## Generate polynomial features
    X = polynomial_features(x, order)
    ## Compute the weights using the normal equation
    w_mle = np.linalg.inv(X.T @ X) @ X.T @ t  ## w = (X^T*X)^(-1)*X^(T)*t, because the cost function ||Xw - y||^2 
    return w_mle

def map_regression(x, t, order, alpha):
    """ Perform MAP regression using polynomial features """
    X = polynomial_features(x, order)
    ## Adjust the normal equation to include the regularization term
    regularized_matrix = X.T @ X + alpha * np.eye(order + 1)
    w_map = np.linalg.inv(regularized_matrix) @ X.T @ t
    return w_map

## Function to predict new values
def mle_predict(x, w):
    """ Predict t using the MLE model """
    X_new = polynomial_features(x, ORDER)
    t_pred = X_new @ w
    return t_pred

# Bayesian Regression Function
def bayesian_regression(x, t, order, alpha, beta):
    """ Perform Bayesian regression """
    X = polynomial_features(x, order)
    S_N_inv = alpha * np.eye(order + 1) + beta * (X.T @ X)
    S_N = np.linalg.inv(S_N_inv)
    m_N = beta * S_N @ X.T @ t
    return m_N, S_N

# Predictive Distribution
def predictive_distribution(x_new, m_N, S_N, beta):
    """ Calculate predictive mean and variance """
    phi_new = polynomial_features(x_new, ORDER)
    mean = phi_new @ m_N
    variance = 1 / beta + np.sum(phi_new @ S_N * phi_new, axis=1)
    return mean, variance

In [ ]:
## Constants
BETA = 11.1 ## Beta is a precision for the likelihood function
ALPHA = 5e-3 ## alpha is a precision for prior distribution
NUM_SAMPLES = 100
ORDER = 9

## Generate sample points: using random sampling instead of linspace to get i.i.d samples
np.random.seed(0)  ## For reproducibility
x_samples = np.random.uniform(-1, 1, NUM_SAMPLES)
## Calculating the target values t with noise
t_samples = np.cos(2 * np.pi * x_samples) + np.sin(np.pi * x_samples) + np.random.normal(0, 1 / np.sqrt(BETA), NUM_SAMPLES)

In [ ]:
## Fit MLE model
w_mle = mle_regression(x_samples, t_samples, ORDER)
## Fit MAP model
w_map = map_regression(x_samples, t_samples, ORDER, ALPHA)
## Bayesian Regression
m_N, S_N = bayesian_regression(x_samples, t_samples, ORDER, ALPHA, BETA)
## Test the model with new samples
x_test = np.linspace(-1, 1, 100)
means, variances = predictive_distribution(x_test, m_N, S_N, BETA)


## 참고한 사이트

https://euphoria0-0.github.io/posts/Bayesian-Linear-Regression/

https://github.com/zjost/bayesian-linear-regression/blob/master/src/bayes-regression.ipynb

https://code-first-ml.github.io/book1/notebooks/introduction/mle_coin.html